<a href="https://colab.research.google.com/github/alberttigga/ragexampleapp/blob/main/Talk_with_text_File.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install langchain huggingface_hub sentence_transformers faiss-cpu unstructured chromadb Cython tiktoken unstructured[local-inference]  groq -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 981.5/981.5 kB 32.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 4.0 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 2.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.4/42.4 kB 2.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 117.0/117.0 kB 6.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.2/42.2 kB 1.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.0/42.0 kB 2.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.5/48.5 kB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 997.8/997.8 kB 36.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━

In [2]:
# Import Colab Secrets userdata module
from google.colab import userdata
import getpass
import os

if "HUGGINGFACEHUB_API_TOKEN" not in os.environ:
   # os.environ["HUGGINGFACEHUB_API_TOKEN"] = getpass.getpass("Provide your HUGGINGFACEHUB TOKEN")
    # Set other API keys similarly
   os.environ["HF_TOKEN"] = userdata.get('HF_TOKEN')

In [16]:
import requests

url = "https://github.com/alberttigga/ragexampleapp/blob/main/Story_Budget_2024_25.txt"
res = requests.get(url)
with open("state_of_the_union.txt", "w") as f:
  f.write(res.text)

In [18]:
!pip install -U langchain-community

In [25]:
# Document Loader
from langchain.document_loaders import TextLoader
loader = TextLoader('./Story_Budget_2024_25.txt', encoding='latin-1') # Try 'latin-1' encoding
documents = loader.load()

In [26]:
documents

[Document(metadata={'source': './Story_Budget_2024_25.txt'}, page_content="Story on Indian Budget 2024-25\n\nThe year was 2024, and the air in New Delhi buzzed with anticipation as the Indian Finance Minister prepared to present the Union Budget for 2024-25. Across the nation, citizens gathered around their televisions and smartphones, eager to hear how the government's financial blueprint would shape the year ahead.\n\nMeera, a small business owner from Pune, anxiously awaited the announcements. Her textile business had been struggling with rising costs and unpredictable market conditions. She hoped for policies that would provide some relief to small entrepreneurs like her.\n\nAs the Finance Minister began the speech, Meera listened intently. The government announced a significant increase in allocations for infrastructure projects, aiming to boost connectivity and create jobs across the country. Meera\x92s hopes rose as she realized that improved logistics could lower her costs and 

In [27]:
import textwrap

def wrap_text_preserve_newlines(text, width=110):
    # Split the input text into lines based on newline characters
    lines = text.split('\n')

    # Wrap each line individually
    wrapped_lines = [textwrap.fill(line, width=width) for line in lines]

    # Join the wrapped lines back together using newline characters
    wrapped_text = '\n'.join(wrapped_lines)

    return wrapped_text

In [28]:
print(wrap_text_preserve_newlines(str(documents[0])))

page_content='Story on Indian Budget 2024-25

The year was 2024, and the air in New Delhi buzzed with anticipation as the Indian Finance Minister prepared
to present the Union Budget for 2024-25. Across the nation, citizens gathered around their televisions and
smartphones, eager to hear how the government's financial blueprint would shape the year ahead.

Meera, a small business owner from Pune, anxiously awaited the announcements. Her textile business had been
struggling with rising costs and unpredictable market conditions. She hoped for policies that would provide
some relief to small entrepreneurs like her.

As the Finance Minister began the speech, Meera listened intently. The government announced a significant
increase in allocations for infrastructure projects, aiming to boost connectivity and create jobs across the
country. Meeras hopes rose as she realized that improved logistics could lower her costs and expand her
market reach.

The Minister then shifted focus to taxation,

In [29]:
# Text Splitter
from langchain.text_splitter import CharacterTextSplitter
text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
docs = text_splitter.split_documents(documents)

In [30]:
len(docs)

2

In [31]:
docs[1]

Document(metadata={'source': './Story_Budget_2024_25.txt'}, page_content='The Minister then shifted focus to taxation, announcing a reduction in the corporate tax rate for small businesses and increased incentives for startups. Meera felt a wave of relief wash over her. The tax cuts would allow her to invest in new machinery and hire additional workers, helping her business grow.\n\nThe budget also emphasized green energy initiatives, with substantial funding directed toward solar power and electric vehicles. Meera, ever the entrepreneur, saw an opportunity to diversify into eco-friendly textiles, a growing market.\n\nAs the speech concluded, Meera felt a renewed sense of optimism. The 2024-25 budget, with its focus on infrastructure, small business support, and sustainability, was a beacon of hope. It wasn\x92t just a plan on paper; it was a lifeline that promised to breathe new life into her business and, by extension, into the millions of dreams carried by entrepreneurs across India

Embedding texts

In [32]:
# Embeddings
from langchain.embeddings import HuggingFaceEmbeddings

embeddings = HuggingFaceEmbeddings()

/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:151: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 0.3.0. An updated version of the class exists in the langchain-huggingface package and should be used instead. To use it run `pip install -U langchain-huggingface` and import as `from langchain_huggingface import HuggingFaceEmbeddings`.
  warn_deprecated(
/usr/local/lib/python3.10/dist-packages/sentence_transformers/cross_encoder/CrossEncoder.py:11: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.6k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/363 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [33]:
# Vectorstore: https://python.langchain.com/en/latest/modules/indexes/vectorstores.html
from langchain.vectorstores import FAISS

db = FAISS.from_documents(docs, embeddings)

In [34]:
query = "Budget emphasizes in which initiatives?"
docs = db.similarity_search(query)

In [40]:
print(wrap_text_preserve_newlines(str(docs[0].page_content)))

The Minister then shifted focus to taxation, announcing a reduction in the corporate tax rate for small
businesses and increased incentives for startups. Meera felt a wave of relief wash over her. The tax cuts
would allow her to invest in new machinery and hire additional workers, helping her business grow.

The budget also emphasized green energy initiatives, with substantial funding directed toward solar power and
electric vehicles. Meera, ever the entrepreneur, saw an opportunity to diversify into eco-friendly textiles, a
growing market.

As the speech concluded, Meera felt a renewed sense of optimism. The 2024-25 budget, with its focus on
infrastructure, small business support, and sustainability, was a beacon of hope. It wasnt just a plan on
paper; it was a lifeline that promised to breathe new life into her business and, by extension, into the
millions of dreams carried by entrepreneurs across India.


QA Chain

In [41]:
from langchain.chains.question_answering import load_qa_chain
from langchain import HuggingFaceHub
from langchain_community.llms import HuggingFaceEndpoint

In [42]:
llm=HuggingFaceEndpoint(repo_id="mistralai/Mistral-7B-Instruct-v0.2", temperature=0.1, max_length=512)

                    max_length was transferred to model_kwargs.
                    Please make sure that max_length is what you intended.


In [43]:
chain = load_qa_chain(llm, chain_type="stuff")

In [44]:
query = "Budget emphasizes in which all other initiatives than solar power?"
docs = db.similarity_search(query)
chain.run(input_documents=docs, question=query)

' The budget also emphasized green energy initiatives, specifically electric vehicles.'

In [45]:
query = "What is the focus in 2024-25 budget?"
docs = db.similarity_search(query)
chain.run(input_documents=docs, question=query)

' The 2024-25 Indian Union Budget focuses on infrastructure development, providing relief to small businesses through tax cuts and incentives, and promoting green energy initiatives.'